In [1]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import datapane as dp
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import datetime
import matplotlib.pyplot as plt
import plotly.io as pio
import fbprophet
from plotly import tools
from fbprophet.plot import plot_plotly, plot_components_plotly

py.init_notebook_mode(connected=True)

In [2]:
df = pd.read_excel('Sample - Superstore.xls')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
daily_quantity = df.groupby('Order Date')['Sales'].sum().reset_index()

trace0 = go.Scatter(x=daily_quantity['Order Date'], y=df['Sales'], name='nflx')
fig0 = go.Figure([trace0])
fig0.update_layout(
    title={
        'text': "Total Sales by Day",
        'x':0.5,
        'xanchor': 'center'})
fig0.show()

In [4]:
df['year'] = df['Order Date'].dt.year
df['month'] = df['Order Date'].dt.month
df['dow'] = df['Order Date'].dt.dayofweek
df['day'] = df['Order Date'].dt.day

trace0 = go.Scatter(x=df.groupby('year')['Profit'].sum().index, y=df.groupby('year')['Profit'].sum().values)
trace1 = go.Scatter(x=df.groupby('month')['Profit'].sum().index, y=df.groupby('month')['Profit'].sum().values)
trace2 = go.Scatter(x=df.groupby('dow')['Profit'].sum().index, y=df.groupby('dow')['Profit'].sum().values)
trace3 = go.Scatter(x=df.groupby('day')['Profit'].sum().index, y=df.groupby('day')['Profit'].sum().values)

fig1 = tools.make_subplots(rows=4, cols=1, subplot_titles=('Total Profit by Year', 'Total Profit by Month', 'Total Profit by Day of Week', 'Total Profit by Day of Month'))
fig1.append_trace(trace0, 1,1)
fig1.append_trace(trace1, 2,1)
fig1.append_trace(trace2, 3,1)
fig1.append_trace(trace3, 4,1)
fig1['layout'].update(title='', showlegend=False)   

/Users/kaan.simsek/anaconda3/envs/analytics/lib/python3.8/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Total Profit by Year',
                     'x': 0.5,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Total Profit by Month',
                     'x': 0.5,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 0.71875,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'Total Profit by Day of Week',
                     'x': 0.5,
                     'xanchor': 'center',
                     'xref': 'paper',
           

In [5]:
trace1 = go.Bar(
    x=df[df.Segment=='Consumer'].groupby('month')['Sales'].sum().index,
    y=df[df.Segment=='Consumer'].groupby('month')['Sales'].sum().values,
    name = 'Consumer',
    marker=dict(
        color = ('aqua')
    )
)

trace2 = go.Bar(
    x=df[df.Segment=='Corporate'].groupby('month')['Sales'].sum().index,
    y=df[df.Segment=='Corporate'].groupby('month')['Sales'].sum().values,
    name='Corporate',
    marker=dict(
        color = ('red')
    )
)

trace3 = go.Bar(
    x=df[df.Segment=='Home Office'].groupby('month')['Sales'].sum().index,
    y=df[df.Segment=='Home Office'].groupby('month')['Sales'].sum().values,
    name='Home Office',
    marker=dict(
        color = ('navy')
    )
)

layout = go.Layout(
    title={
        'text': "Total Sales by Segment",
        'x':0.5,
        'xanchor': 'center'}, xaxis = dict(title = 'Month')
)
data = [trace1,trace2,trace3]
fig2 = go.Figure(data=data, layout=layout)
fig2.show()

In [6]:
trace1 = go.Bar(
    x=df[df.Segment=='Consumer'].groupby('month')['Profit'].sum().index,
    y=df[df.Segment=='Consumer'].groupby('month')['Profit'].sum().values,
    name='Consumer',
    marker=dict(
        color = ('aqua')
    ),
)

trace2 = go.Bar(
    x=df[df.Segment=='Corporate'].groupby('month')['Profit'].sum().index,
    y=df[df.Segment=='Corporate'].groupby('month')['Profit'].sum().values,
    name='Corporate',
    marker=dict(
        color = ('red')
    ),
)

trace3 = go.Bar(
    x=df[df.Segment=='Home Office'].groupby('month')['Profit'].sum().index,
    y=df[df.Segment=='Home Office'].groupby('month')['Profit'].sum().values,
    name='Home-Office',
    marker=dict(
        color = ('navy')
    ),
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    title={
        'text': "Total Profit by Segment",
        'x':0.5,
        'xanchor': 'center'}, xaxis = dict(title = 'Month'), barmode='stack'
)

fig3 = go.Figure(data=data, layout=layout)

In [7]:
trace0 = go.Scatter(
    x=df[df.Region=='West'].groupby('month')['Sales'].sum().index,
    y=df[df.Region=='West'].groupby('month')['Sales'].sum().values,
    name = 'West',
    mode='markers',
    marker=dict(
        size=15,
        color = ('aqua')))

trace1 = go.Scatter(
    x=df[df.Region=='East'].groupby('month')['Sales'].sum().index,
    y=df[df.Region=='East'].groupby('month')['Sales'].sum().values,
    name = 'East',
   mode='markers',
    marker=dict(
        size=15,
        color = ('navy')))

trace2 = go.Scatter(
    x=df[df.Region=='Central'].groupby('month')['Sales'].sum().index,
    y=df[df.Region=='Central'].groupby('month')['Sales'].sum().values,
    name = 'Central',
   mode='markers',
    marker=dict(
        size=15,
        color = ('red')))

trace3 = go.Scatter(
    x=df[df.Region=='South'].groupby('month')['Sales'].sum().index,
    y=df[df.Region=='South'].groupby('month')['Sales'].sum().values,
    name = 'South',
   mode='markers',
    marker=dict(
        size=15,
        color = ('lime')))

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    title={
        'text': "Total Sales by Region",
        'x':0.5,
        'xanchor': 'center'}, xaxis = dict(title = 'Month')
)

fig4 = go.Figure(data=data, layout=layout)

In [8]:
trace0 = go.Scatter(
    x=df[df.Region=='West'].groupby('month')['Profit'].sum().index,
    y=df[df.Region=='West'].groupby('month')['Profit'].sum().values,
    name = 'West',
    mode='markers',
    marker=dict(symbol = 'star-square',
        size=15,
        color = ('aqua')))

trace1 = go.Scatter(
    x=df[df.Region=='East'].groupby('month')['Profit'].sum().index,
    y=df[df.Region=='East'].groupby('month')['Profit'].sum().values,
    name = 'East',
   mode='markers',
    marker=dict(symbol = 'star-square',
        size=15,
        color = ('navy')))

trace2 = go.Scatter(
    x=df[df.Region=='Central'].groupby('month')['Profit'].sum().index,
    y=df[df.Region=='Central'].groupby('month')['Profit'].sum().values,
    name = 'Central',
   mode='markers',
    marker=dict(symbol = 'star-square',
        size=15,
        color = ('red')))

trace3 = go.Scatter(
    x=df[df.Region=='South'].groupby('month')['Profit'].sum().index,
    y=df[df.Region=='South'].groupby('month')['Profit'].sum().values,
    name = 'South',
   mode='markers',
    marker=dict(symbol = 'star-square',
        size=15,
        color = ('lime')))

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    title={
        'text': "Total Profit by Region",
        'x':0.5,
        'xanchor': 'center'}, xaxis = dict(title = 'Month')
)

fig5 = go.Figure(data=data, layout=layout)

In [9]:
data = go.Pie(labels=df.groupby('Category')['Sales'].sum().index, 
              values=df.groupby('Category')['Sales'].sum().values,  hole = .2)
layout = go.Layout(
    title={
        'text': "Total Sales by Category",
        'x':0.5,
        'xanchor': 'center'}
)
fig6 = go.Figure(data=data, layout=layout)

In [10]:
data = go.Pie(labels=df.groupby('Sub-Category')['Profit'].sum().index, 
              values=df.groupby('Sub-Category')['Profit'].sum().values,  hole = .2)
layout = go.Layout(
    title={
        'text': "Total Profit by Sub-Category",
        'x':0.5,
        'xanchor': 'center'}, legend=dict(orientation="h")
)
fig7 = go.Figure(data=data, layout=layout)

In [11]:
data = go.Bar(
    y=df.groupby('Category')['Sales'].sum().index,
    x=df.groupby('Category')['Sales'].sum().values,
    marker=dict(
        color = ('red')
    ), orientation='h'
)

layout = go.Layout(
    title={
        'text': "Total Sales by Category",
        'x':0.5,
        'xanchor': 'center'}, legend=dict(orientation="h")
)
fig8 = go.Figure(data=data, layout=layout)

In [12]:
data = go.Bar(
    y=df.groupby('Sub-Category')['Profit'].sum().index,
    x=df.groupby('Sub-Category')['Profit'].sum().values,
    marker=dict(
        color = ('navy')
    ), orientation='h'
)

layout = go.Layout(
    title={
        'text': "Total Profit by Sub-Category",
        'x':0.5,
        'xanchor': 'center'}, legend=dict(orientation="h")
)
fig9 = go.Figure(data=data, layout=layout)

In [13]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,year,month,dow,day
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016,11,1,8
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016,11,1,8
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016,6,6,12
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015,10,6,11
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015,10,6,11


In [14]:
top_cities = df.groupby('City')['Quantity'].sum().sort_values(ascending=False)[:10].index

fv = df[df.City.isin(top_cities)].pivot_table(index="City",columns="dow",values="Sales",
                     aggfunc=lambda x:x.mean())

trace = go.Heatmap(z=fv.values,
                   x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday'],
                   y=fv.index.values, colorscale='oranges', 
                  )

data=[trace]
layout = go.Layout(title={
        'text': "Total Sales by City",
        'x':0.5,
        'xanchor': 'center'})

fig10 = go.Figure(data=data, layout=layout)

In [15]:
top_cities = df.groupby('City')['Quantity'].sum().sort_values(ascending=False)[:10].index

fv = df[df.City.isin(top_cities)].pivot_table(index="City",columns="dow",values="Profit",
                     aggfunc=lambda x:x.mean())

trace = go.Heatmap(z=fv.values,
                   x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday'],
                   y=fv.index.values, colorscale='Greens', 
                  )

data=[trace]
layout = go.Layout(title={
        'text': "Total Profit by City",
        'x':0.5,
        'xanchor': 'center'})

fig11= go.Figure(data=data, layout=layout)

In [16]:
forecasting_data = df.groupby('Order Date')['Profit'].sum().reset_index()
forecasting_data = forecasting_data.rename(columns={'Order Date': 'ds', 'Profit': 'y'})
m = fbprophet.Prophet()
m.fit(forecasting_data)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [17]:
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

In [18]:
fig12=plot_plotly(m, forecast, trend=True)
fig12.update_layout(autosize=True, width=None)

In [19]:
fig13=plot_components_plotly(m, forecast, figsize=(1500,200))
fig13.update_layout(autosize=True, width=None)

In [20]:
dp.Report(

    "# Data exploration",

    dp.Blocks(

        dp.Plot(fig0, caption='This plot shows total sales by day from 2014 to 2017.'), 

        dp.Plot(fig1, caption='This plot shows total profit by year, month, day of week, day of month.'), 

        dp.Plot(fig2, caption='This plot shows total sales by consumer, corporate, home-office segments.'), 

        dp.Plot(fig3, caption='This plot shows total profit by consumer, corporate, home-office segments.'), 

        dp.Plot(fig4, caption='This plot shows total sales by west, east, central, south regions.'), 

        dp.Plot(fig5, caption='This plot shows total profit by west, east, central, south regions.'), 

        dp.Plot(fig6, caption='This plot shows total sales by category.'), 

        dp.Plot(fig7, caption='This plot shows total profit by sub-category.'),

        dp.Plot(fig8, caption='This plot shows total sales by category.'),

        dp.Plot(fig9, caption='This plot shows total profit by sub-category.'),

        dp.Plot(fig10, caption='This plot shows total sales by city.'),

        dp.Plot(fig11, caption='This plot shows total profit by city.'),

        columns=2,

        rows=6

    ), 

    "# Forecasting ",

    dp.Plot(fig12, caption='This plot shows daily profit forecasting using fbprophet.'),

      dp.Plot(fig13, caption='This plot shows trend, yearly seasonality, and weekly seasonality of the time series.'),

    full_width=True

).publish(description='',
          name='ecommerce_report', open=True)

Publishing report and associated data - please wait..
Report successfully published at https://datapane.com/u/kaan/reports/ecommerce-report/
